In [6]:
#Importing Lib
import pandas as pd
import numpy as np
import numpy_financial as npf
import warnings
from scipy.optimize import newton
from datetime import datetime
from pandas.tseries.offsets import DateOffset


warnings.filterwarnings('ignore')
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500

In [7]:
#Setting IRR 
folder='79th_buy'
irr_target = 8.05

In [8]:
#Setting date (running on thursday)

#for p date1
from datetime import datetime, timedelta
today = datetime.today()
days_until_tuesday = (1 - today.weekday() + 7) % 7  # 1 is Tuesday
days_until_tuesday = 7 if days_until_tuesday == 0 else days_until_tuesday
next_tuesday = today + timedelta(days=days_until_tuesday)
pdate=next_tuesday.strftime('%Y-%m-%d')


#for sfy and buy

curr_date=datetime.today()
curr_date=curr_date.strftime('%m-%d-%Y')

#for FX file 
today = datetime.today()
first_day_of_current_month = datetime(today.year, today.month, 1)
fd=first_day_of_current_month.strftime('%Y-%m-%d')
last_day_previous_month = first_day_of_current_month - timedelta(days=1)
last_end= f"{last_day_previous_month.year}_{last_day_previous_month.month:03}_{last_day_previous_month.day:02}"



print("for pdate=",pdate)
print("for curr date=",curr_date)
print("for fx file=",last_end)
print("first date of month for fx files",fd)

for pdate= 2025-11-18
for curr date= 11-14-2025
for fx file= 2025_010_31
first date of month for fx files 2025-11-01


In [9]:
#manual dates
pdate='2025-11-18'
curr_date='11-13-2025'
last_end='2025_0010_31'
fd='2025-11-01'

In [10]:
#file location and importing

#files_required
fx3_servicing_file = f"{folder}/files_required/FX3_{last_end}.xlsx"
fx4_servicing_file = f"{folder}/files_required/FX4_{last_end}.xlsx"
#fx4_servicing_file = f"C:/Users/gdehankar/TWG/Jagadish Balaji - SFC_Loans/78th_buy/files_required/FX4_2025_0010_31.xlsx"
loans = pd.read_csv(f"{folder}/files_required/Tape20Loans_{(pd.to_datetime('today') - pd.DateOffset(days = 3)).strftime("%m-%d-%Y")}.csv")
df_loans_types = pd.read_excel(f'{folder}/files_required/MASTER_SHEET.xlsx')
notes = pd.read_excel(f'{folder}/files_required/MASTER_SHEET - Notes.xlsx')
existing_file_ = pd.read_csv(f"{folder}/files_required/current_assets.csv") #do not change
sfy_file = f"{folder}/files_required/FX3_{curr_date}_ExhibitAtoFormofSaleNotice.xlsx" 
prime_file = f"{folder}/files_required/11-13-2025 Exhibit A To Form Of Sale Notice.xlsx"

#underwriting
# underwriting_sfy = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY')
# underwriting_prime = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime')
# underwriting_sfy_notes = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY - Notes')
# underwriting_prime_notes = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime - Notes')
# sfy_comap = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY COMAP')
# sfy_comap2 = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'SFY COMAP2')
# prime_comap = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime COMAP')
# prime_comap_new = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Prime CoMAP - New')
# notes_comap = pd.read_excel(f"{folder}/files_required/Underwriting_Grids_COMAP.xlsx", sheet_name = 'Notes CoMAP')

In [11]:
sfy_df = pd.read_excel(sfy_file)
print(sfy_df)

         Service Finance Company, LLC.     Unnamed: 1  \
0     Exhibit A to Form of Sale Notice            NaN   
1                        Loan Schedule            NaN   
2                     Date: 11/13/2025            NaN   
3             Closing Date: 11/18/2025            NaN   
4                                Count  SELLER Loan #   
...                                ...            ...   
1005                              1001    SFC_6392423   
1006                              1002    SFC_6393931   
1007                              1003    SFC_6394207   
1008                              1004    SFC_6395071   
1009                               NaN            NaN   

                   Unnamed: 2           Unnamed: 3     Unnamed: 4  \
0                         NaN                  NaN            NaN   
1                         NaN                  NaN            NaN   
2                         NaN                  NaN            NaN   
3                         NaN          

In [12]:
prime_df = pd.read_excel(prime_file)
print(prime_df)

        Service Finance Company, LLC.     Unnamed: 1               Unnamed: 2  \
0    Exhibit A to Form of Sale Notice            NaN                      NaN   
1                       Loan Schedule            NaN                      NaN   
2                    Date: 11/13/2025            NaN                      NaN   
3            Closing Date: 11/18/2025            NaN                      NaN   
4                               Count  SELLER Loan #               Asset Type   
..                                ...            ...                      ...   
510                               506    SFC_6392533           HVAC EQUIPMENT   
511                               507    SFC_6392958  HVAC-PLUMBING EQUIPMENT   
512                               508    SFC_6393507           HVAC EQUIPMENT   
513                               509    SFC_6394521           HVAC EQUIPMENT   
514                               NaN            NaN                      NaN   

              Unnamed: 3   

In [13]:
sfy_df = pd.read_excel(sfy_file)
sfy_df = sfy_df.iloc[4:].reset_index(drop=True)
sfy_df.columns = sfy_df.iloc[0]
sfy_df = sfy_df[1:].reset_index(drop=True)
sfy_df = sfy_df.iloc[:-1].reset_index(drop=True)
print(sfy_df)

0    Count SELLER Loan #                   Asset Type          Submit Date  \
0        1   SFC_5172657  BATH TUB AND SHOWER SYSTEMS  2024-09-18 21:22:00   
1        2   SFC_5515050                WINDOWS/DOORS  2025-02-04 19:25:00   
2        3   SFC_5534066                       SIDING  2025-02-12 11:13:00   
3        4   SFC_5566778                WINDOWS/DOORS  2025-02-24 19:54:00   
4        5   SFC_5568589                      WINDOWS  2025-02-25 14:49:00   
...    ...           ...                          ...                  ...   
999   1000   SFC_6391276      HVAC-PLUMBING EQUIPMENT  2025-11-06 17:55:00   
1000  1001   SFC_6392423             ROOFING MATERIAL  2025-11-07 10:55:00   
1001  1002   SFC_6393931               HVAC EQUIPMENT  2025-11-07 15:26:00   
1002  1003   SFC_6394207      HVAC-PLUMBING EQUIPMENT  2025-11-07 16:15:00   
1003  1004   SFC_6395071      HVAC-PLUMBING EQUIPMENT  2025-11-07 19:51:00   

0    Orig. Balance Purchase Price Current Balance    APR Term P

In [14]:
prime_df = pd.read_excel(prime_file)
prime_df = prime_df.iloc[4:].reset_index(drop=True)
prime_df.columns = prime_df.iloc[0]
prime_df = prime_df[1:].reset_index(drop=True)
prime_df = prime_df.iloc[:-1].reset_index(drop=True)
print(prime_df)

0   Count SELLER Loan #               Asset Type          Submit Date  \
0       1   SFC_5491676            WINDOWS/DOORS  2025-01-27 10:48:00   
1       2   SFC_5591626            WINDOWS/DOORS  2025-03-05 12:56:00   
2       3   SFC_5663341                  WINDOWS  2025-03-27 23:02:00   
3       4   SFC_5672553            WINDOWS/DOORS  2025-03-31 15:20:00   
4       5   SFC_5674400            WINDOWS/DOORS  2025-03-31 20:18:00   
..    ...           ...                      ...                  ...   
504   505   SFC_6391444       PLUMBING EQUIPMENT  2025-11-06 18:36:00   
505   506   SFC_6392533           HVAC EQUIPMENT  2025-11-07 11:20:00   
506   507   SFC_6392958  HVAC-PLUMBING EQUIPMENT  2025-11-07 12:36:00   
507   508   SFC_6393507           HVAC EQUIPMENT  2025-11-07 14:15:00   
508   509   SFC_6394521           HVAC EQUIPMENT  2025-11-07 17:15:00   

0   Orig. Balance Purchase Price Current Balance    APR Term  \
0         23811.3       23025.53         23811.3   19.5   6

In [15]:
#df_loans_types
df_loans_types["Platform"] = df_loans_types.platform.str.upper()

#notes
notes['loan program'] = notes['loan program'].apply(lambda x: x + 'notes')
notes["Platform"] = notes.platform.str.upper()

#df_loans_types
df_loans_types = pd.concat([df_loans_types, notes])

#existing_file_
existing_file_['Submit Date'] = pd.to_datetime(existing_file_['Submit Date'])
existing_file_['Purchase_Date'] = pd.to_datetime(existing_file_['Purchase_Date'])
existing_file_['Monthly Payment Date'] = pd.to_datetime(existing_file_['Monthly Payment Date'])
existing_file = existing_file_.copy()
del existing_file_

print(existing_file.shape)
existing_file.Purchase_Date.value_counts().sort_index()

(176152, 58)


Purchase_Date
2024-04-30    3629
2024-05-10    1410
2024-05-17    1660
2024-05-24    1815
2024-05-31    1577
2024-06-10    1307
2024-06-17    1663
2024-06-25    1745
2024-06-28    1426
2024-07-15    2892
2024-07-22    2631
2024-07-31    3387
2024-08-05    1475
2024-08-12    2655
2024-08-19    2313
2024-08-30    4470
2024-09-10    2108
2024-09-17    2254
2024-09-24    1637
2024-09-30    2090
2024-10-08    2391
2024-10-15    2126
2024-10-22    1849
2024-10-29    1796
2024-11-05    2093
2024-11-12    2057
2024-11-19    1907
2024-11-26    1888
2024-12-03    1770
2024-12-10    2106
2024-12-17    2144
2024-12-24    1600
2024-12-31    1886
2025-01-07    1444
2025-01-14    2035
2025-01-21    1970
2025-01-28    1889
2025-02-04    2014
2025-02-11    2157
2025-02-18    1951
2025-02-25    1922
2025-03-04    2052
2025-03-11    1834
2025-03-18    1916
2025-03-25    1970
2025-04-01    1896
2025-04-09    2352
2025-04-15    2399
2025-04-22    2779
2025-04-29    2826
2025-05-06    2847
2025-05-13    291

In [16]:
#For analysis ignore
import pandas as pd
def analyze_dataset(df):
    # Separate numeric and character columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    character_cols = df.select_dtypes(include=['object', 'category']).columns

    # Analyze numeric columns
    print("=== Numeric Column Analysis ===")
    for col in numeric_cols:
        print(f"\nColumn: {col}")
        print(f"Total Count: {df[col].count()}")
        print(f"Missing Values: {df[col].isnull().sum()}")
        print(f"Mean: {df[col].mean()}")
        print(f"Median: {df[col].median()}")
        for p in range(10, 100, 10):
            print(f"{p}th Percentile: {df[col].quantile(p / 100)}")

    # Analyze character columns
    print("\n=== Character Column Analysis ===")
    for col in character_cols:
        print(f"\nColumn: {col}")
        print(f"Total Count: {df[col].count()}")
        print(f"Missing Values: {df[col].isnull().sum()}")
        print(df[col].value_counts())

analyze_dataset(df_loans_types)

=== Numeric Column Analysis ===

Column: promo_term
Total Count: 368
Missing Values: 0
Mean: 5.532608695652174
Median: 0.0
10th Percentile: 0.0
20th Percentile: 0.0
30th Percentile: 0.0
40th Percentile: 0.0
50th Percentile: 0.0
60th Percentile: 3.0
70th Percentile: 6.0
80th Percentile: 10.0
90th Percentile: 12.0

Column: servicing_cost
Total Count: 368
Missing Values: 0
Mean: 0.018516304347826088
Median: 0.02
10th Percentile: 0.0165
20th Percentile: 0.0165
30th Percentile: 0.0165
40th Percentile: 0.0165
50th Percentile: 0.02
60th Percentile: 0.02
70th Percentile: 0.02
80th Percentile: 0.02
90th Percentile: 0.02

Column: irr_support
Total Count: 368
Missing Values: 0
Mean: 0.012076086956521737
Median: 0.0125
10th Percentile: 0.0115
20th Percentile: 0.0115
30th Percentile: 0.0115
40th Percentile: 0.0115
50th Percentile: 0.0125
60th Percentile: 0.0125
70th Percentile: 0.0125
80th Percentile: 0.0125
90th Percentile: 0.0125

Column: late_percent
Total Count: 368
Missing Values: 0
Mean: 2e-0

In [17]:
loans['tagging'] = loans['Loan Group'].apply(lambda x:'SFY' if 'FX3' in x or 'FX1' in x else 'PRIME')
loans['Account Number'] = loans['Account Number'].astype(int)
loans['Status Codes'].fillna("", inplace = True)
loans['Repurchased'] = loans['Status Codes'].apply(lambda x: True if 'REPURCHASE' in [i.strip() for i in x.split(";")] else False)
loans['Open Date'] = pd.to_datetime(loans['Open Date'])
loans['maturityDate'] = pd.to_datetime(loans['maturityDate'])
print(loans['Repurchased'].value_counts())
loans['SELLER Loan #'] = loans['Account Number'].apply(lambda x:"SFC_" + str(x))
repurchased_loans = loans[loans['Repurchased'] == True]['SELLER Loan #'].values

existing_file.reset_index(drop = True, inplace = True)
existing_file.loc[existing_file['SELLER Loan #'].isin(repurchased_loans), 'Repurchase'] = True 

print(repurchased_loans.shape)

Repurchased
False    173906
True        846
Name: count, dtype: int64
(846,)


In [18]:
sfy_df['Platform'] = "SFY"
prime_df['Platform'] = "PRIME"
buy_df = pd.concat([prime_df, sfy_df])
buy_df.rename(columns = {"Loan Program" : "loan program"}, inplace = True)
buy_df['loan program'] = buy_df.apply(lambda x: x['loan program'] + 'notes' if x['Application Type'] == 'HD NOTE' else x['loan program'], axis = 1)
buy_df["Repurchase"] = False
buy_df["Repurchase_Date"] = None
buy_df["Purchase_Date"] = pd.to_datetime(pdate)
buy_df["Excess_Asset"] = False
buy_df["Borrowing_Base_eligible"] = True
buy_df["IRR Support Target"] = irr_target
buy_df['Submit Date'] = pd.to_datetime(buy_df['Submit Date'])
buy_df['Purchase_Date'] = pd.to_datetime(buy_df['Purchase_Date'])
buy_df['Monthly Payment Date'] = pd.to_datetime(buy_df['Monthly Payment Date'])
buy_df = buy_df.merge(df_loans_types, on = ['loan program', 'Platform'], how = 'left')
assert sum(buy_df['SELLER Loan #'].duplicated()) == 0
#buy_df.loc[(buy_df['new_programs'] == True)&(buy_df['Repurchase'] == False) & (buy_df['platform'] == 'sfy'), 'Excess_Asset'] = True 
#buy_df.loc[(buy_df['new_programs'] == True)&(buy_df['Repurchase'] == False) & (buy_df['platform'] == 'sfy'), 'Borrowing_Base_eligible'] = False 
# buy_df['Dealer Fee'] = buy_df['Dealer Fee'] / 100
final_df = pd.concat([buy_df, existing_file[existing_file['Purchase_Date'] > '2025-09-30']])
final_df_all = pd.concat([buy_df, existing_file])
buy_df.shape, final_df[final_df['Orig. Balance'] > 50000].shape, final_df.shape, final_df['SELLER Loan #'].nunique(), buy_df[buy_df['tagging'] != 'BD']['Orig. Balance'].sum()

((1513, 63), (117, 64), (10469, 64), 10469, 20799862.519999985)

## Mark excess and borrowing file

In [19]:
final_df_all[["SELLER Loan #", 'Orig. Balance', "tagging", 'Repurchase', 
              'platform']].to_excel(f"{folder}/output/EveryLoan.xlsx")

In [20]:
# del rkdf, rkdf1, rkdf2
rkdf1 = pd.read_excel(fx3_servicing_file)
rkdf2 = pd.read_excel(fx4_servicing_file)
rkdf = pd.concat([rkdf1.iloc[:-1], rkdf2.iloc[:-1]])
rkdf['SELLER Loan #'] = rkdf['Trial Balance'].apply(lambda x: "SFC_" + str(int(x)))
rkdf['Current Purchase Price'] = (rkdf["Principal Balance"] * rkdf["PP%"]) / 100
print(final_df[final_df["SELLER Loan #"].isin(rkdf["SELLER Loan #"])].shape, rkdf.shape)
rkdf = rkdf[rkdf["SELLER Loan #"].isin(final_df["SELLER Loan #"])].copy()
rkdf.sort_values("SELLER Loan #", inplace = True)
rkdf.head(2)

(5950, 64) (74466, 30)


,Trial Balance,Short Name,Portfolio,Status,Status Code,Group,Class1,Class2,Open Date,Maturity Date,Last Payment Date,Next Interest Due Date,Next Principal Due Date,Expiration Date,Loan Interest Rate,Current Perdiem,Past Due Days,Original Amount,Principal Balance,Interest Balance,Fees Balance,Late Charge Balance,Total Balance,Suspense Balance,PP%,State,Tranche,Unnamed: 27,SELLER Loan #,Current Purchase Price
119,5108785,-,WPDI,ACTIVE,NaN,FX4-In,20 Mth WPDI,Oct 2025,10/8/2025,10/8/2030,10/28/2025,12/7/2025,12/7/2025,6/8/2027,23.99,8.602032,0,13600.00,13087.71,25.80,0.0,0.0,13113.519630,0.0,93.63,OH,NaN,,SFC_5108785,12254.022873
222,5205054,-,ALT,ACTIVE,NaN,FX4,Unsec Std - 1290 - 180,Oct 2025,10/21/2025,10/21/2040,10/21/2025,12/20/2025,12/20/2025,NaN,12.90,11.607070,0,32841.71,32841.71,127.67,0.0,0.0,32969.387771,0.0,100.42,MN,NaN,,SFC_5205054,32979.645182


In [21]:
# del still_exist, paid, check_final_df, repurchased
final_df.reset_index(drop = True, inplace = True)
final_df.sort_values("SELLER Loan #", inplace = True)
still_exist = final_df[final_df["SELLER Loan #"].isin(rkdf['SELLER Loan #'])].copy()
still_exist.sort_values("SELLER Loan #", inplace = True)
still_exist["Current Balance"] = rkdf["Principal Balance"].values
still_exist["Purchase Price"] = rkdf["Current Purchase Price"].values

paid = final_df[(~final_df["SELLER Loan #"].isin(list(rkdf['SELLER Loan #']))) & (final_df.Purchase_Date < fd)].copy()
paid["Current Balance"] = 0
paid["Purchase Price"] = 0

rest = final_df[~final_df['SELLER Loan #'].isin(list(still_exist['SELLER Loan #']) + list(paid['SELLER Loan #']) )].copy()
check_final_df = pd.concat([still_exist, paid, rest])
check_final_df.loc[check_final_df.Repurchase == True, 'Current Balance'] = 0
check_final_df.loc[check_final_df.Repurchase == True, 'Purchase Price'] = 0
check_final_df.shape, final_df.shape

((10469, 64), (10469, 64))

In [22]:
still_exist.shape, paid.shape, check_final_df.shape

((5950, 64), (63, 64), (10469, 64))

In [23]:
check_final_df['Orig. Balance'].sum(), final_df['Orig. Balance'].sum(), check_final_df['Current Balance'].sum(), final_df['Current Balance'].sum()

(150772566.15000018, 150772566.15000007, 149510096.2500002, 150772566.15000007)

In [24]:

check_final_df.reset_index(drop = True, inplace = True)

check_final_df[["Purchase_Date","SELLER Loan #","tagging","Platform","Excess_Asset","Borrowing_Base_eligible",'Orig. Balance',"Purchase Price","Current Balance","Repurchase",
                "Application Type"]].to_excel(f"{folder}/output/borrowing_file.xlsx")

## Check Purchase Price

In [25]:
existing_file['purchase_price_check'] = existing_file['Lender Price(%)'] == round(existing_file['modeled_purchase_price']*100, 2)
existing_file.shape, existing_file.purchase_price_check.sum()

((176152, 59), np.int64(176152))

In [26]:
buy_df['purchase_price_check'] = buy_df['Lender Price(%)'] == round(buy_df['modeled_purchase_price']*100, 2)
buy_df.shape, buy_df.purchase_price_check.sum()

((1513, 64), np.int64(1496))

In [27]:
final_df['purchase_price_check'] = final_df['Lender Price(%)'] == round(final_df['modeled_purchase_price']*100, 2)
final_df.shape, final_df.purchase_price_check.sum()

((10469, 65), np.int64(10452))

## Underwriting_checks

In [28]:
# del underwriting_sfy, underwriting_prime

loan_ids = []
min_oncome = []
for row in buy_df[buy_df['Application Type'] != 'HD NOTE'].iterrows():
    if row[1]['platform'] == 'sfy':
        underwriting = underwriting_sfy.copy()
    else:
        underwriting = underwriting_prime.copy()
    if row[1]['SELLER Loan #'] in tuloans or row[1]['SELLER Loan #'] in buy_df[buy_df['Application Type'] == 'HD NOTE']['SELLER Loan #'].values:
        continue
    prog = row[1]['loan program']
    mth_income = row[1]['Income'] / 12
    fico = row[1]['FICO Borrower']
    dti = row[1]['DTI']
    pti = row[1]['PTI']
    reason = ""
    balance = row[1]['Orig. Balance'] - row[1]['Stamp fee']
    # (underwriting.approval_high >= balance) & (underwriting.approval_low <= balance)
    filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.monthly_income_min <= mth_income) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
    meet_crit = False
    for i in filter_one.iterrows():
        if balance <= i[1]['approval_high'] and round(dti*100,0) <= i[1]['dti_max']:
            meet_crit = True
    if not meet_crit:
        if fico > 700:
            filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
            for i in filter_one.iterrows():
                if balance <= i[1]['approval_high'] and round(dti*100, 0) <= i[1]['dti_max'] and pti <= i[1]["pti_ratio"]:
                    meet_crit = True
            if meet_crit:
                min_oncome.append(row[1]['SELLER Loan #'])
    if not meet_crit:
        loan_ids.append(row[1]['SELLER Loan #'])
    del underwriting
    # break

len(loan_ids), len(min_oncome)

NameError: name 'underwriting_prime' is not defined

In [29]:
# del underwriting_sfy_notes, underwriting_prime_notes

loan_ids_notes = []
min_oncome_notes = []
for row in buy_df[buy_df['Application Type'] == 'HD NOTE'].iterrows():
    if row[1]['platform'] == 'sfy':
        underwriting = underwriting_sfy_notes.copy()
    else:
        underwriting = underwriting_prime_notes.copy()
    if row[1]['SELLER Loan #'] in tuloans:
        continue
    prog = row[1]['loan program'].replace('notes', '')
    mth_income = row[1]['Income'] / 12
    fico = row[1]['FICO Borrower']
    dti = row[1]['DTI']
    pti = row[1]['PTI']
    reason = ""
    balance = row[1]['Orig. Balance'] - row[1]['Stamp fee']
    (underwriting.approval_high >= balance) & (underwriting.approval_low <= balance)
    filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.monthly_income_min <= mth_income) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
    meet_crit = False
    for i in filter_one.iterrows():
        if balance <= i[1]['approval_high'] and round(dti*100,0) <= i[1]['dti_max']:
            meet_crit = True
    if not meet_crit:
        if fico > 700:
            filter_one = underwriting[(underwriting.finance_type_name_nls == prog) & (underwriting.fico_min <= fico)].sort_values("approval_high").reset_index(drop = True)
            for i in filter_one.iterrows():
                if balance <= i[1]['approval_high'] and round(dti*100, 0) <= i[1]['dti_max'] and pti <= i[1]["pti_ratio"]:
                    meet_crit = True
            if meet_crit:
                min_oncome_notes.append(row[1]['SELLER Loan #'])
    if not meet_crit:
        loan_ids_notes.append(row[1]['SELLER Loan #'])
    del underwriting
    # break

len(loan_ids_notes), len(min_oncome_notes)

NameError: name 'underwriting_prime_notes' is not defined

In [30]:
prime_comap_cols_min_fico = {j:int(j[:3]) for i,j in enumerate(['660-699', '700-739', '740-749', '750-769', '770+'])}
sfy_comap_cols_min_fico = {j:int(j[:3]) for i,j in enumerate(['660-719', '720-779', '780-799', '800+'])}
sfy_comap_cols_min_fico2 = {j:int(j[:3]) for i,j in enumerate(['660-699', '700-739', '740-749', '750-769','770+'])}
notes_comap_cols_min_fico = {j:int(j[:3]) for i,j in enumerate(['680-749', '750-769', '770-789', '790+'])}

In [31]:
print(buy_df)

     Count SELLER Loan #               Asset Type         Submit Date  \
0        1   SFC_5491676            WINDOWS/DOORS 2025-01-27 10:48:00   
1        2   SFC_5591626            WINDOWS/DOORS 2025-03-05 12:56:00   
2        3   SFC_5663341                  WINDOWS 2025-03-27 23:02:00   
3        4   SFC_5672553            WINDOWS/DOORS 2025-03-31 15:20:00   
4        5   SFC_5674400            WINDOWS/DOORS 2025-03-31 20:18:00   
...    ...           ...                      ...                 ...   
1508  1000   SFC_6391276  HVAC-PLUMBING EQUIPMENT 2025-11-06 17:55:00   
1509  1001   SFC_6392423         ROOFING MATERIAL 2025-11-07 10:55:00   
1510  1002   SFC_6393931           HVAC EQUIPMENT 2025-11-07 15:26:00   
1511  1003   SFC_6394207  HVAC-PLUMBING EQUIPMENT 2025-11-07 16:15:00   
1512  1004   SFC_6395071  HVAC-PLUMBING EQUIPMENT 2025-11-07 19:51:00   

     Orig. Balance Purchase Price Current Balance    APR Term  \
0          23811.3       23025.53         23811.3   19.5  

In [ ]:
loan_not_in_comap = []
for row in buy_df[(buy_df['Application Type'] != 'HD NOTE') & (buy_df['purchase_price_check'] == True)].iterrows():
    fico = row[1]['FICO Borrower']
    prog = row[1]['loan program']
    found = False
    if row[1]['platform'] == 'prime':
        if prog not in prime_comap[['660-699', '700-739', '740-749', '750-769', '770+']].stack().unique():
            continue
        if row[1]['Submit Date'] > pd.to_datetime('2024-06-11'):
            for col in ['660-699', '700-739', '740-749', '750-769', '770+']:
                if prog in prime_comap_new[col].values and fico >= prime_comap_cols_min_fico[col]:
                    found = True
                    break
        else:
            for col in ['660-699', '700-739', '740-749', '750-769', '770+']:
                if prog in prime_comap[col].values and fico >= prime_comap_cols_min_fico[col]:
                    found = True
                    break
        if not found:
            # reason = f"PRIME loan prog not in fico band - 660-699 - available - {fico}"
            loan_not_in_comap.append((row[1]["SELLER Loan #"], prog, "PRIME"))
    else:
        if prog in sfy_comap[['660-719', '720-779', '780-799', '800+']].stack().unique():
            for col in ['660-719', '720-779', '780-799', '800+']:
                if prog in sfy_comap[col].values and fico >= sfy_comap_cols_min_fico[col]:
                    found = True
                    break
            if not found:
                # reason = f"PRIME loan prog not in fico band - 660-699 - available - {fico}"
                loan_not_in_comap.append((row[1]["SELLER Loan #"], prog, "SFY"))
        elif prog in sfy_comap2[['660-699', '700-739', '740-749', '750-769','770+']].stack().unique():
            for col in ['660-699', '700-739', '740-749', '750-769','770+']:
                if prog in sfy_comap2[col].values and fico >= sfy_comap_cols_min_fico2[col]:
                    found = True
                    break
            if not found:
                # reason = f"PRIME loan prog not in fico band - 660-699 - available - {fico}"
                loan_not_in_comap.append((row[1]["SELLER Loan #"], prog, "SFY"))
        else:
            continue
            
len(loan_not_in_comap)

In [ ]:
loan_not_in_comap_notes = []
for row in buy_df[(buy_df['Application Type'] == 'HD NOTE') & (buy_df['purchase_price_check'] == True)].iterrows():
    fico = row[1]['FICO Borrower']
    prog = row[1]['loan program']
    found = False
    if prog not in notes_comap[['680-749', '750-769', '770-789', '790+']].stack().unique():
        continue
    for col in ['680-749', '750-769', '770-789', '790+']:
        if prog in notes_comap[col].values and fico >= notes_comap_cols_min_fico[col]:
            found = True
            break
    if not found:
        # reason = f"PRIME loan prog not in fico band - 660-699 - available - {fico}"
        loan_not_in_comap_notes.append((row[1]["SELLER Loan #"], prog, "NOTES"))
            
len(loan_not_in_comap_notes)    

In [ ]:
#FOR Ourself
final_df[final_df['purchase_price_check'] == False].to_excel(f"{folder}/output/purchase_price_mismatch.xlsx")
buy_df[buy_df["SELLER Loan #"].isin(loan_ids)].to_excel(f"{folder}/output/flagged_loans_{folder}.xlsx")
buy_df[buy_df["SELLER Loan #"].isin(loan_ids_notes)].to_excel(f"{folder}/output/NOTES_flagged_loans_{folder}.xlsx")
buy_df[buy_df["SELLER Loan #"].isin([i[0] for i in loan_not_in_comap])].to_excel(f"{folder}/output/comap_not_passed.xlsx")

In [ ]:
#sharing file generation
purchase=final_df[final_df['purchase_price_check'] == False]
flag=buy_df[buy_df["SELLER Loan #"].isin(loan_ids)]
note_flag=buy_df[buy_df["SELLER Loan #"].isin(loan_ids_notes)]
comap=buy_df[buy_df["SELLER Loan #"].isin([i[0] for i in loan_not_in_comap])]
purchase = purchase.iloc[:, :30]
flag = flag.iloc[:, :30]
note_flag = note_flag.iloc[:, :30]
comap = comap.iloc[:, :30]
purchase.to_excel(f"{folder}/output_share/purchase_price_mismatch.xlsx")
flag.to_excel(f"{folder}/output_share/flagged_loans_{folder}.xlsx")
note_flag.to_excel(f"{folder}/output_share/NOTES_flagged_loans_{folder}.xlsx")
comap.to_excel(f"{folder}/output_share/comap_not_passed.xlsx")

## Eligibility Checks

#### Check A

In [ ]:
final_df_all[(final_df_all.Repurchase == False) & (final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.Repurchase == False)&(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all['SELLER Loan #'] != 'SFC_4731527') & (final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term <= 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Purchase Price'].shape[0] / final_df_all[(final_df_all.platform == 'prime')]['Purchase Price'].shape[0]

#### Check B

In [ ]:
final_df_all[(final_df_all.Repurchase == False) & (final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() 

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].shape[0] / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].shape[0]

#### Check C

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.Term > 144) & (final_df_all.type == 'standard') & (final_df_all['FICO Borrower'] >= 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

#### Check D

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'hybrid')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

#### Check E

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'ninp')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

#### Check F

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'epni')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

#### Check G

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all.type == 'wpdi')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

#### Check H

In [ ]:
final_df_all[(final_df_all.platform == 'prime')]['Lender Price(%)'].max()

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all['Lender Price(%)'] > 100) & (final_df_all['Lender Price(%)'] <= 103)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
np.sum((final_df_all['Dealer Fee'] * final_df_all['Orig. Balance'])/final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum())

#### Check I

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all['Orig. Balance'] > 50000)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].mean()

#### Check J

In [ ]:
(pd.pivot_table(final_df_all[(final_df_all.platform == 'prime')], values = 'Orig. Balance', aggfunc = 'sum', columns='Property State') / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()).T.sort_values("Orig. Balance", ascending = False)

#### Check L

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all['FICO Borrower'] < 680)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'prime') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum()

In [ ]:
np.sum((final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'] * final_df_all[(final_df_all.platform == 'prime')]['FICO Borrower'] ) / final_df_all[(final_df_all.platform == 'prime')]['Orig. Balance'].sum())

In [ ]:
final_df_all[(final_df_all.platform == 'prime')]['FICO Borrower'].mean()

### SFY

#### Check A

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'hybrid')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'hybrid') & (final_df_all.APR < 7.0)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

#### Check B

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp') & (final_df_all.promo_term > 6)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp') & (final_df_all.promo_term > 12)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'ninp') & (final_df_all.Term > 84)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

#### Check C

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'epni')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

#### Check D

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'wpdi')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'wpdi', 'wpdi_bd']))]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'wpdi') & (final_df_all.promo_term >= 12)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'wpdi', 'wpdi_bd'])) & (final_df_all.promo_term >= 12)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

#### Check E

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'standard') & (final_df_all.Term > 120)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'standard', 'standard_bd'])) & (final_df_all.Term > 120)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'standard') & (final_df_all.Term > 144)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type.isin([ 'standard', 'standard_bd'])) & (final_df_all.Term > 144)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

#### Check F

In [ ]:
final_df_all[(final_df_all.platform == 'sfy')]['Lender Price(%)'].max()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['Lender Price(%)'] > 100) & (final_df_all['Lender Price(%)'] <= 103)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['Lender Price(%)'] > 100) & (final_df_all['Lender Price(%)'] <= 103) & (final_df_all['loan program'] != "Unsec Std - 999 - 120")]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
np.sum(final_df_all['Dealer Fee'] * final_df_all['Orig. Balance'])/final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

#### Check G

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['Orig. Balance'] > 50000)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].mean()

#### Check H

In [ ]:
(pd.pivot_table(final_df_all[(final_df_all.platform == 'sfy')], values = 'Orig. Balance', aggfunc = 'sum', columns='Property State') / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()).T.sort_values("Orig. Balance", ascending = False)

#### Check J

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['FICO Borrower'] < 680)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all['FICO Borrower'] < 700)]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
np.sum((final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'] * final_df_all[(final_df_all.platform == 'sfy')]['FICO Borrower'] ) / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum())

In [ ]:
final_df_all[(final_df_all.platform == 'sfy')]['FICO Borrower'].mean()

#### Check L

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'wpdi_bd')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.platform == 'sfy') & (final_df_all.type == 'standard_bd')]['Orig. Balance'].sum() / final_df_all[(final_df_all.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
final_df_all[(final_df_all.type.isin( ['standard_bd', 'wpdi_bd']))]['Purchase Price'].sum() / final_df_all['Purchase Price'].sum()

In [ ]:
final_df_all[(final_df_all.type.isin( ['standard_bd', 'wpdi_bd'])) & (final_df_all.Excess_Asset != True)]['Purchase Price'].sum() / final_df_all['Purchase Price'].sum()

In [ ]:
buy_df[(buy_df.platform == 'sfy') & (buy_df.type == 'standard_bd')]['Orig. Balance'].sum() / buy_df[(buy_df.platform == 'sfy')]['Orig. Balance'].sum()

In [ ]:
with pd.ExcelWriter('Ratios.xlsx') as writer:
    pd.pivot_table(df_loans_types, index="tagging", columns = "type", values = 'weights', aggfunc='sum').to_excel(writer, sheet_name = 'Ideal Distribution')
    (pd.pivot_table(final_df_all[final_df_all['Repurchase'] == False], index="tagging", columns = "type", values = 'Orig. Balance', aggfunc='sum')/final_df_all[final_df_all['Repurchase'] == False]["Orig. Balance"].sum()).to_excel(writer, sheet_name = 'All Buys (Orig. Balance)')
    (pd.pivot_table(final_df_all[final_df_all['Repurchase'] == False], index="tagging", columns = "type", values = 'Purchase Price', aggfunc='sum')/final_df_all[final_df_all['Repurchase'] == False]["Purchase Price"].sum()).to_excel(writer, sheet_name = 'All Buys (Purcahse Price)')
    (pd.pivot_table(buy_df, index="tagging", columns = "type", values = 'Orig. Balance', aggfunc='sum')/buy_df["Orig. Balance"].sum()).to_excel(writer, sheet_name = '15th Buy (Orig. Balance)')
    (pd.pivot_table(buy_df, index="tagging", columns = "type", values = 'Purchase Price', aggfunc='sum')/buy_df["Purchase Price"].sum()).to_excel(writer, sheet_name = '15th Buy (Purcahse Price)')